# Word Segmentation Modelling (version : alam.1.0)

This is the modelling documentation of Word Segmentation

First Structured :

    ModelLstm(
    (lstm): LSTM(26, 25)
    (linear): Linear(in_features=25, out_features=15, bias=False)
    (sigmoid): Sigmoid()
    )


Input :

    INPUT LENGTH =15
    TRAIN = 90k data (Random state : 342)
    



output :


## Plan

    [*] Get Data
    [*] Transform the data
    [ ] Prepare the model
    [ ] Train it

In [1]:
import sys
sys.path.append('../scr/')
import pandas as pd
import torch
import numpy as np
import os
import pickle
import WordSegmenTools as wst
from torch.autograd import Variable


In [2]:
PATH_DATA_RAW='../../data/raw/'
PATH_DATA_CLN='../../data/clean/'
PATH_MODEL='../../data/model/'

In [3]:
os.listdir(PATH_DATA_CLN)

['data_clean_100k.res']

## Get Data

In [4]:
INPUT_LENGTH=15

In [5]:
# data_cln=pd.read_csv(PATH_DATA_CLN+'data_clean_100k.res')
data_cln=pickle.load(open(PATH_DATA_CLN+'data_clean_100k.res','rb'))

In [6]:
data_cln['len_source']=data_cln.source.apply(len)
data_used=data_cln[data_cln.len_source==INPUT_LENGTH]
data_used=data_used.reset_index()[['source','target','index']]
data_used.columns=['source','target','ids']

In [7]:
data_used.sample(10)

,source,target,ids
15288,sayapuaspraktek,saya puas praktek,1471818
68664,inggrisrackakan,inggris rack akan,1724894
90218,gerindraawalnya,gerindra awalnya,1827426
2965,valverdeterukur,valverde terukur,1414002
6016,golbanyaksiaran,gol banyak siaran,1428192
18800,kemiliariniluar,ke miliar ini luar,1488527
5,venilagaberusia,veni laga berusia,1400012
5495,danputaranastra,dan putaran astra,1425866
89630,perusahaankamis,perusahaan kamis,1824516
99639,liputanyangbisa,liputan yang bisa,1872289


## Transform Data

In [8]:
data_used['flag_space'] = data_used['target'].map(wst.get_flag_space)
word2idx, idx2word = wst.get_label_index(data_used.source)

In [9]:
data_used_train=data_used.sample(90000,random_state=342)
data_used_test=data_used[~(data_used.ids.isin(data_used_train.ids))]

In [10]:
## Target 
Y_tr=np.array(data_used_train.flag_space.apply(list).tolist(),dtype=np.double)
Y_te=np.array(data_used_test.flag_space.apply(list).tolist(),dtype=np.double)

## Source
X_tr=wst.char_vectorizer(data_used_train.source.tolist(), word2idx, INPUT_LENGTH)
X_te=wst.char_vectorizer(data_used_test.source.tolist(), word2idx, INPUT_LENGTH)

In [11]:
X_tr.shape

(90000, 15, 26)

## Prepare the Model

In [12]:
## LSTM Example
class ModelLstm(torch.nn.Module):

    def __init__(self, input_size, output_size, hidden_dim):        
        super(ModelLstm, self).__init__()
        self.input_dim = input_size
        self.hidden_dim = hidden_dim

        # self.lstm = torch.nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers)
        self.lstm = torch.nn.LSTM(self.input_dim, self.hidden_dim)
        self.linear = torch.nn.Linear(self.hidden_dim, output_size,bias=False)
        self.sigmoid = torch.nn.Sigmoid()

    # Why do sometimes need hidden init sometimes dont?
    def init_hidden(self,embed_len,batch):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(embed_len, batch, self.hidden_dim),
               torch.zeros(embed_len, batch, self.hidden_dim))

    def forward(self, input):
        torch.manual_seed(200)
        batch_size=len(input)
        embedding_len=len(input[0][0])        
#         input=input.view(len(input[0]), batch_size, -1)
        input=input.view(batch_size,len(input[0]), -1)
        
        
        lstm_out, hidden2 = self.lstm(input)
        # lstm_out,h = self.lstm(input)
        
        # Only take the output from the final timetep
        out = lstm_out[:,-1]        
        out = out.view(batch_size,-1)
        y_pred = self.linear(out)
        y_pred = self.sigmoid(y_pred)
        
        return y_pred,'-'
    
## Remember : Batch, Seq, Embedding

In [13]:
def data_batching_torch(X,y,batch):
    batches=[(torch.from_numpy(X[i*(batch):i*(batch)+batch]).type('torch.DoubleTensor'),torch.from_numpy(y[i*(batch):i*(batch)+batch]).type('torch.DoubleTensor'))   
                  for i in range(int(len(X)/batch)+1)]
    return batches

In [14]:
BATCH_SIZE=35
training_samples=data_batching_torch(X_tr,Y_tr,BATCH_SIZE)
len(training_samples)

2572

In [15]:
# Get the model
outputDim = 1        # takes variable 'y'
hidden = 25
learningRate = 0.006
EMBED_SIZE=len(word2idx)
model = ModelLstm(input_size=EMBED_SIZE,output_size=INPUT_LENGTH,hidden_dim=hidden)
model=model.double()

##### For GPU #######
if torch.cuda.is_available():
    model.cuda()
    
## Input Length == Output Length

In [16]:
## Choose the optimizer and criterion
# criterion = torch.nn.CrossEntropyLoss()
criterion = torch.nn.BCELoss()

# optimizer = torch.optim.Adam(model.parameters(),lr=learningRate)
optimizer = torch.optim.SGD(model.parameters(),lr=learningRate)

# optimizer = torch.optim.LBFGS(model.parameters(), lr=learningRate)


In [17]:
AA=training_samples[0][1]

In [18]:
AA.type()

'torch.DoubleTensor'

## Train it

In [19]:
epochs=9
running_loss_train = 0
model.train()
for epoch in range(epochs):
    running_loss=[]
    # Converting inputs and labels to Variable
    
    for batch_idx, (data, target) in enumerate(training_samples, 1):
        if data.size()[0]>0:
            inputs = Variable(data)
            labels = Variable(target)
            optimizer.zero_grad()
            outputs, hidden = model(inputs)
            # get loss for the predicted output
            loss = criterion(outputs, labels)
            loss.backward()
            # update parameters
            optimizer.step()

            running_loss.append(loss.item())
            running_loss_train += (loss.item() - running_loss_train) / batch_idx
        
    if epoch % 3 == 0:
        print("running loss train", running_loss_train)

running loss train 0.6488855424531341
running loss train 0.3322530512277687
running loss train 0.32280282163075286


In [20]:
model.eval()

ModelLstm(
  (lstm): LSTM(26, 25)
  (linear): Linear(in_features=25, out_features=15, bias=False)
  (sigmoid): Sigmoid()
)

In [21]:
## Training
XX=X_tr.copy()
XX=Variable(torch.from_numpy(XX).type('torch.DoubleTensor'))
yy=model(XX)

In [33]:
yy0=yy[0].data.numpy()
yy0=(yy0>0.5).astype(int).astype(str)
yy0=[''.join(i) for i in yy0]
yy0[:10]

['000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000']

In [34]:
data_used_train['y_pred']=yy0

In [36]:
data_used_train.sample(10)

,source,target,ids,flag_space,y_pred
50144,melainkandaerah,melainkan daerah,1637189,000000001000000,000000000000000
67640,menegaskantiket,menegaskan tiket,1719973,000000000100000,000000000000000
87884,pelakusetiapini,pelaku setiap ini,1815936,000001000001000,000000000000000
32800,tahunberbobotdi,tahun berbobot di,1554907,000010000000100,000000000000000
60884,terpopulerdalam,terpopuler dalam,1687901,000000000100000,000000000000000
15182,identikinisudah,identik ini sudah,1471301,000000100100000,000000000000000
65497,denganpekerjaan,dengan pekerjaan,1709857,000001000000000,000000000000000
48114,mobiltahunsedan,mobil tahun sedan,1627587,000010000100000,000000000000000
58791,terkaitterhadap,terkait terhadap,1678152,000000100000000,000000000000000
31506,sertatariftahap,serta tarif tahap,1548878,000010000100000,000000000000000


## Test it

In [37]:
## Test it
XX=X_te.copy()
XX=Variable(torch.from_numpy(XX).type('torch.DoubleTensor'))
yy=model(XX)

In [38]:
yy0=yy[0].data.numpy()
yy0=(yy0>0.5).astype(int).astype(str)
yy0=[''.join(i) for i in yy0]
yy0[:10]

['000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000',
 '000000000000000']

In [39]:
data_used_test['y_pred']=yy0

/Users/alamhanz/anaconda/envs/word_s/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [40]:
data_used_train.sample(10)

,source,target,ids,flag_space,y_pred
12342,sepakperusahaan,sepak perusahaan,1457750,000010000000000,000000000000000
9138,miliarmelakukan,miliar melakukan,1442854,000001000000000,000000000000000
3403,asalpendaftaran,asal pendaftaran,1416167,000100000000000,000000000000000
74342,jelasnyastadium,jelasnya stadium,1751813,000000010000000,000000000000000
12597,cmnphargaplakat,cmnp harga plakat,1459043,000100001000000,000000000000000
86940,bahwadenganyang,bahwa dengan yang,1811573,000010000010000,000000000000000
54344,terjadibniingin,terjadi bni ingin,1657235,000000100100000,000000000000000
8989,untuktetappiper,untuk tetap piper,1442103,000010000100000,000000000000000
15108,ratelalusamping,rate lalu samping,1471008,000100010000000,000000000000000
28477,danyangnokiadwi,dan yang nokia dwi,1534286,001000100001000,000000000000000
